In [1]:
import synapseclient
from synapseclient import File
syn = synapseclient.Synapse()

syn.login()

Welcome, Lorena Pantano!


In [2]:

entity = syn.get('syn2324928')


Downloaded   [####################]100.00%     82.1MB/82.1MB hm450_gene_annotations.tsv Done...
    

probes = {}
with open(entity.path)) as in_handle:
    with open("../../data/hm450_gene_annotations.bed", "w") as out_handle:
        h = in_handle.next().replace("\"","").strip().split("\t")
        for line in in_handle:            
            c = line.strip().replace("\"","").split("\t")
            probes[c[0]] = c
            print >>out_handle, "\t".join(c[1:4] + [c[0],".","+"] )
          
            

from pybedtools import BedTool

meth = BedTool("hm450_gene_annotations.bed")
mirna = BedTool("/home/lpantano/orch/groups/seqcluster/data/annotation/hsa/hsa.gff3")

ann = meth.intersect(mirna,wo=True)

for line in ann:
    if line[8] == "miRNA_primary_transcript":
        precursor = line[14].split(";")[2].split("=")[1]
        probes[line[3]]=probes[line[3]] + [precursor]
    
with open("../../data/hm450_gene_annotations_w_mirna.tsv", "w") as out_handle:
    for p in probes:
        print >>out_handle, "\t".join(probes[p])
            

trying to map directly

In [3]:
from collections import defaultdict
mirbase = {}
synm = defaultdict(set)
with open("../../data/mirbase/miRNA.str") as in_handle:
    for line in in_handle:
        if line.startswith(">hsa"):
            c = line.strip().split()
            pre = c[0][1:]
            m1 = c[2][1:].split(":")[0]
            m2 = ""
            if len(c) > 3:
                m2 = c[3][1:].split(":")[0]
            mirbase[pre] = [m1, m2]
            syn_name_base = "".join(pre.split("-")[1:3]).upper()
            # if pre.find("mir-203") > -1:
            #    print syn_name_base.endswith("A") or syn_name_base.endswith("B") or syn_name_base.endswith("C")
            synm[syn_name_base].add(pre)
            number = pre.split("-")
            if len(number) > 3:
                syn_name = syn_name_base + "-" + number[3]
                synm[syn_name].add(pre)
                syn_name = syn_name_base + number[3]
                synm[syn_name].add(pre)
                syn_name = syn_name_base[-1] + number[3]
                synm[syn_name].add(pre)
            if syn_name_base.endswith("A") or syn_name_base.endswith("B") or syn_name_base.endswith("C"):
                synm[syn_name_base[:-1]].add(pre)
                if len(number) > 3:
                    syn_name = syn_name_base[:-1] + "-" + number[3]
                    synm[syn_name].add(pre)
                    syn_name = syn_name_base[:-1] + number[3]
                    synm[syn_name].add(pre)
            
print synm["LET7A1"]
print synm["MIR129-1"]
print synm["MIR1270"]

set(['hsa-let-7a-1'])
set(['hsa-mir-129-1'])
set(['hsa-mir-1270'])


In [4]:
new_file = "../../data/hm450_gene_annotations_mapped_mirna.tsv"
with open(entity.path) as in_handle:
    with open(new_file, "w") as out_handle:

        h = in_handle.next().replace("\"","").strip().split("\t")
        for line in in_handle:            
            c = line.strip().replace("\"","").split("\t")
            find = []
            if c[14].find("MIR") > -1:
                names = [n for n in c[14].split(";") if n.startswith("MIR")]
                new_names = []
                for n in names:
                    if n.find("LET") > -1:
                        n = n.replace("MIR","")
                    n = n.replace("HG","")
                    if n.startswith("LET") and len(n)>6:
                        for p in range(4,len(n)):
                            new_names.append(n[:4] + n[p])
                    else:
                        new_names.append(n)
                find = []
                for n in new_names:
                    
                    if n in synm:
                        find.extend(list(synm[n]))
                    elif n.endswith("-1"):
                        if n[:-2] in synm:
                            find.extend(list(synm[n[:-2]]))
                    elif n[:-1] in synm:
                        find.extend(list(synm[n[:-1]]))
                    if n=="MIR3545":
                        if "MIR203B" in synm:
                            find.extend(list(synm["MIR203B"]))
                    if not find:
                        print n
                        raise ValueError(n)
                for p in find:
                    for m in mirbase[p]:
                        if m:
                            print >>out_handle, line.split()[0][1:-1] + "\t" + m


In [6]:
new_entity = File(new_file, description='Methylation miRNA mapping', parent="syn2499689", used='syn2324928')
# new_entity.activity.used('syn2324928')
new_entity = syn.store(new_entity)


##################################################
 Uploading file to Synapse storage 
##################################################
Uploaded Chunks [####################]100.00%     513.6kB/513.6kB ../../data/hm450_gene_annotations_mapped_mirna.tsv Done...
Upload completed in 3 seconds.


In [7]:
new_entity

File(modifiedOn=u'2015-08-20T14:51:07.032Z', contentType=None, createdOn=u'2015-08-20T14:51:07.032Z', md5=None, versionUrl=u'/repo/v1/entity/syn4895962/version/1', synapseStore=True, fileSize=None, used=[u'syn2324928'], createdBy=u'3324792', versions=u'/repo/v1/entity/syn4895962/version', versionNumber=1, uri=u'/repo/v1/entity/syn4895962', cacheDir='../../data', externalURL=None, uri=u'/entity/syn4895962/annotations', versionLabel=u'1', entityType=u'org.sagebionetworks.repo.model.FileEntity', parentId=u'syn2499689', id=u'syn4895962', concreteType='org.sagebionetworks.repo.model.FileEntity', files=['hm450_gene_annotations_mapped_mirna.tsv'], description=u'Methylation miRNA mapping', modifiedBy=u'3324792', creationDate=u'1440082267032', annotations=u'/repo/v1/entity/syn4895962/annotations', etag=u'a1e30a94-1937-422e-86d1-aae4638f1506', dataFileHandleId=u'5105234', path='../../data/hm450_gene_annotations_mapped_mirna.tsv', accessControlList=u'/repo/v1/entity/syn4895962/acl', name=u'hm450_